# Защита данных клиентов

Необходимо защитить данные клиентов. Для этого нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные так, чтобы при их преобразовании качество моделей машинного обучения не ухудшилось. 

**Набор данных:**
 - **Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
 - **Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

data = pd.read_csv('/datasets/insurance.csv')

Подготовим данные, если это потребуется. Для начала посмотрим на них.

In [2]:
display(data.head(15))
display(data.describe())
display(data.info())
data.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


Данные в порядке. Выбросов нет, пропусков тоже. 

Проверим на дубликаты.

In [3]:
data.duplicated().sum()

153

Целых 153 дубликата. Удалим их.

In [4]:
data = data.drop_duplicates().reset_index(drop = True)
display(data.duplicated().sum())
data.info()

0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
Пол                  4847 non-null int64
Возраст              4847 non-null float64
Зарплата             4847 non-null float64
Члены семьи          4847 non-null int64
Страховые выплаты    4847 non-null int64
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


Отлично. Едем дальше.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** признаки умножают на обратимую матрицу, изменится ли качество линейной регрессии? (Её можно обучить заново)

Проверим это, проделав ряд алгебраических операций, чтобы дать обоснованный ответ на этот вопрос.

Нужно доказать верность выражения $
a = a\_new 
$
, где:
$$
a = X(X^T X)^{-1} X^T y
$$
$$
a\_new = XP((XP)^T XP)^{-1} (XP)^T y
$$

Преобразуем переменную `a_new`, применяя свойства транспонированных и обратных матриц, а также свойства их умножения.

**Обоснование:** 
$$
a\_new = XP((XP)^T XP)^{-1} (XP)^T y =
$$
$$
= XP((P^T X^T) XP)^{-1} P^T X^T y = 
$$
$$
= XP(P^T (X^T XP))^{-1} P^T X^T y =
$$
$$
= XP(X^TXP)^{-1} (P^T)^{-1} P^T X^T y =
$$
$$
= XP(X^TXP)^{-1} X^T y =
$$
$$
= XP((X^TX)P)^{-1} X^T y =
$$
$$
= XP P^{-1}(X^TX)^{-1} X^T y =
$$
$$
= X(X^TX)^{-1} X^T y
$$
<br>
<br>
**Итог:**
$$
a\_new = X(X^TX)^{-1} X^T y
$$
$$
a = X(X^T X)^{-1} X^T y
$$

$$
a = a\_new 
$$
<br>
**Ответ:** нет, качество линейной регрессии не изменится, поскольку при домножении признаков на обратимую матрицу получаются предсказания идентичные изначальным.

## Алгоритм преобразования

**Алгоритм**

Шифрование данных клиентов с помощью ключа, в качестве которого у нас используется произвольная обратимая (а, значит, квадратная) матрица. Если дословно, то мы домножаем нашу матрицу на обратимую каждый раз, когда она встречается в выражении. 

Зная нашу произвольную матрицу, мы можем как зашифровать, так и расшифровать сведения.

**Обоснование**

Пунктом выше мы доказали, что при данном подходе наши предсказания меняться не будут, а соответственно и качество модели не пострадает. 

## Проверка алгоритма

Создадим нашу произвольную обратимую (квадратную) матрицу.

In [5]:
# Перед этим выделим целевой признак, например, "страховые выплаты"
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']

# Создадим матрицу
m = features.shape[1]# ее размерность m*m (квадратная)
state = np.random.RandomState(12345)
r = state.normal(size = (m,m))
print('Наша произвольная матрица')
print( )
print(r)
print( )

# Проверим на обратимость
np.linalg.inv(r)

Наша произвольная матрица

[[-0.20470766  0.47894334 -0.51943872 -0.5557303 ]
 [ 1.96578057  1.39340583  0.09290788  0.28174615]
 [ 0.76902257  1.24643474  1.00718936 -1.29622111]
 [ 0.27499163  0.22891288  1.35291684  0.88642934]]



array([[-1.31136747,  0.3921804 ,  0.18868055, -0.67088287],
       [ 1.75872714,  0.14106138, -0.17773045,  0.79787127],
       [-0.41702659, -0.22854768,  0.3550602 ,  0.33039819],
       [ 0.58912996,  0.19073027, -0.5545481 ,  0.6259302 ]])

Матрица обратимая, все будет работать должным образом.

Проведем проверку на метрике R2, качесто линейной регрессии не должно отличаться до и после преобразования.

In [6]:
# Напишем класс для создания модели линейной регрессии с собственными методами
class LinearRegression:
    def fit(self, features, target):
        X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
        y = target
        w = np.linalg.inv(X.T.dot(X)) @ (X.T) @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, features):
        return features.dot(self.w) + self.w0

    
# Качество линейной регрессии до шифрования
model_1 = LinearRegression()
model_1.fit(features, target)
predictions = model_1.predict(features)
print('R2 до   :', r2_score(target, predictions))


# После 
model_2 = LinearRegression()
model_2.fit(features @ r, target)
predictions = model_2.predict(features @ r)
print('R2 после:', r2_score(target, predictions))

R2 до   : 0.4302010044852068
R2 после: 0.4302010044844432


Результаты почти идентичны.<br>
Разность наблюдается, начиная с 10+ знака после запятой. Но это можно скинуть на операции с float, которые накапливают небольшую погрешность.

## Вывод

Шифрование данных клиентов прошло успешно благодаря нами созданному "ключу", в качестве которого мы использовали обратимую матрицу.<br>
Возможность такого преобразования данных мы обосновали выше, доказав последующую неизменность предсказаний.<br>
Отсюда и следствие, что на качество обученной модели такое шифрование тоже не повлияет. Проверили это на практике.

Помним, что в любой момент мы можем расшифровать информацию, зная наш "ключ".